# RAG Exercise: Building a Simple RAG Pipeline

Complete the missing parts (`# TODO`) to build a basic Retrieval-Augmented Generation pipeline using Chroma and OpenAI embeddings + LLM.

## 1. Setup & Imports
Install dependencies and import modules.

In [ ]:
# Install required packages
# !pip install -U chromadb langchain openai

import os
import glob
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

from dotenv import load_dotenv
load_dotenv("../.env")

client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'), base_url='https://api.openai.com/v1')


## 2. Load Documents
Read text files from a folder into a `docs` list of dicts with `text` and `metadata`.

In [ ]:
# TODO: Adjust the path to your docs directory if needed
doc_paths = glob.glob('../docs/*.txt')  
docs = []
for path in doc_paths:
    with open(path, 'r', encoding='utf-8') as f:
        content = f.read()
    docs.append({
        'text': content,
        'metadata': {'source': os.path.basename(path)}
    })

print(f"Loaded {len(docs)} documents.")

## 3. Embedding & Vector Store Ingestion
Create embeddings and ingest documents into Chroma.

In [ ]:
# Initialize embeddings
# TODO: instantiate OpenAIEmbeddings with your API key if needed
txt_emb = OpenAIEmbeddings()  

# TODO: Create a Chroma vector store from texts
vectordb = Chroma.from_texts(
    texts=[d['text'] for d in docs],           # your document texts
    embedding=txt_emb,                          # embedding function
    metadatas=[d['metadata'] for d in docs],    # metadata list
    persist_directory='chroma_db'               # where to persist
)

# Persist to disk
vectordb.persist()
print("Chroma vector store created and persisted.")

## 4. Retrieval Function
Implement a function to retrieve top-k similar documents for a query.

In [ ]:
def retrieve_docs(query: str, k: int = 3):
    """Retrieve top-k documents for the query."""
    # TODO: Use vectordb.similarity_search or as_retriever to get documents
    # Example using similarity_search:
    # results = vectordb.similarity_search(query, k=k)
    # return [doc.page_content for doc in results]
    pass

# Test retrieval
sample_query = "Tell me more about generative AI"
retrieved = retrieve_docs(sample_query, k=2)
print("Retrieved Documents:", retrieved)

## 5. Build RetrievalQA Chain
Initialize the LLM and RetrievalQA, then run a query.

In [ ]:
# TODO: Write your own prompt for question answering using RAG
prompt = ""

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retrieve_docs, "question": RunnablePassthrough()}
    | prompt
    | client
    | StrOutputParser()
)

for chunk in rag_chain.stream("Tell me more about generative AI"):
    print(chunk, end="", flush=True)

## 6. Experiment
- Change `k` in `retrieve_docs` to 1 or 5 and observe differences.
- Try different `chain_type` options: 'stuff', 'map_reduce', 'refine'.
- Try different `prompts` in the `rag_chain`